# Теоремы об исключении и продолжении

## Упражнения к параграфу 1

### Упражнение 2

Рассмотрим систему уравнений $$x^2 + 2y^2 = 3$$
                         $$x^2 + xy + y^2 = 3$$
(a) Пусть $I = (x^2 + 2y^2 - 3, x^2 + xy + y^2 - 3)$ - соответствующий идеал.
Найти базисы идеалов $I\cap k[x]$ и $I\cap k[y]$.

(b) Найти все решения системы.

(c) Определить, какие из этих решений рациональны.

(d) Найти наименьшее поле $k$ такое, что все решения принадлежат $k^2$.

**Решение.**

По **теореме об исключении** базис $G_1$ идеала $I_1 = I\cap k[x]$ равен $G \cap k[x]$, где $G$ - базис Грёбнера идеала $I$.

Тогда для начала найдём базис Грёбнера $G$ относительно лексикографического порядка $y > x$. Для этого нам понадобится функция $groebner$ из библиотеки $SymPy$.

In [60]:
from sympy.polys.polytools import groebner
from sympy.abc import x, y

F = [y**2 + x*y + x**2 - 3, 2*y**2 + x**2 - 3]           # Идеал I
G = groebner(F, y, x, order='lex')                       # Базис Грёбнера g

print('\nБазис Грёбнера относительно y > x:')

def print_groebner(G):
    if (len(G) > 0):
        for i in range(len(G.exprs)):
            print('g', i+1, ' = ', G.exprs[i], sep='')   # метод .exprs позволяет записать полином в виде красивого выражения
    else:
        print('is empty!')

print_groebner(G)



Базис Грёбнера относительно y > x:
g1 = x**3 - 3*x + 2*y
g2 = x**4 - 4*x**2 + 3


Теперь несложно найти $G_1 = G \cap k[x]$:

In [61]:
from sympy import Poly, LC
from sympy import solve, solve_poly_system

poly_system = G.exprs

def first_exclusion_step(var, poly_system, *args):
    G_ex = []
    I_next = []
    index = 1
    for pol in poly_system:
        f = Poly(pol, *var)                                  # Представим каждое выражение как полином от переменной X
        if f.free_symbols_in_domain.isdisjoint(set(args)):   # Если помимо переменной X в полиноме НЕТ свободной переменной Y,
            G_ex.append((f.as_expr(), index))                # то этот полином содержится в G1
        else:
            I_next.append((Poly(f, args), index))            # Запомним отдельно полиномы, которые не содержатся в G1.
        index += 1
    return G_ex, I_next

G1, I0 = first_exclusion_step([x], poly_system, y)

print('\nБазис Грёбнера 1-ого исключающего идеала:')

def print_groebner_of_ex(G):
    if (len(G) > 0):
        for pol in G:
            print('g', pol[1], ' = ', pol[0], sep='')
    else:
        print('is empty!')

print_groebner_of_ex(G1)



Базис Грёбнера 1-ого исключающего идеала:
g2 = x**4 - 4*x**2 + 3


Аналогично найдём $G1 = G \cap k[y]$. Для этого нам нужен лексикографический порядок $x > y$.

In [62]:
F = [x**2 + 2*y**2 - 3, x**2 + x*y + y**2 - 3]           # Идеал I
G = groebner(F, x, y, order='lex')                       # Базис Грёбнера G

print('\nБазис Грёбнера относительно x > y:')
print_groebner(G)

poly_system = G.exprs
G1, I0 = first_exclusion_step([y], poly_system, x)

print('\nБазис Грёбнера 1-ого исключающего идеала:')     
print_groebner_of_ex(G1)



Базис Грёбнера относительно x > y:
g1 = x**2 + 2*y**2 - 3
g2 = x*y - y**2
g3 = y**3 - y

Базис Грёбнера 1-ого исключающего идеала:
g3 = y**3 - y


Заметим, что многочлен $g_3$ зависит только от одной переменной $y$. Мы сделали **шаг исключения**, избавившись от переменной $x$. Решив это уравнение, получим возможные значения $y$ для решения всей системы - это будет **частичным решением**.

По **теореме о продолжении** следующий шаг - шаг продолжения - может не выполниться (т.е. частичное решение не верно для всей системы), если коэффициенты при старших мономах одновременно равны нулю. Сделаем проверку на это для остальных полиномов, не содержащихся в $G_1$

In [63]:
roots_y = []
poly_system = []
for pol in G1:
    f_y = Poly(pol[0], y)
    poly_system.append(f_y)
roots_y = solve_poly_system(poly_system, y)               # Нашли частичные решения функцией solve_poly_system()
print('корни Y:', roots_y)
    
for root_y in roots_y:
    for pol in I0:
        f_x = Poly(pol[0], x)
        if root_y not in solve(LC(f_x), y):               # Проверка, обращает ли в ноль частичное решение старшие мономы
            break                                         # полиномов из I0
    else:
        roots_y.remove(root_y)
print('корни Y после проверки:', roots_y)


корни Y: [(-1,), (0,), (1,)]
корни Y после проверки: [(-1,), (0,), (1,)]


Теперь сделаем **шаг продолжения**. Найдём все решения системы и сразу определим, какие из решений рациональны.

In [64]:
# Найдём значения X:
roots_x = []
for root_y in roots_y:
    poly_system = []                                           # Для каждого частичного решения Y будут свои решения X
    for pol in I0:
        poly_system.append(pol[0].subs(y, root_y[0]))          # Подставим значения частичных решений методом .subs(y,root)
    solution = solve_poly_system(poly_system, x)               # и решим систему уравнений от одной переменной X
    roots_x.append(solution)

# Теперь мы знаем все решения системы, "закинем" их красиво в список решений solutions:
solutions = []
for i in range(len(roots_y)):
    for root_x in roots_x[i]:
        solutions.append((root_x[0], roots_y[i][0]))
print('Все решения системы: ')
print(*solutions, sep='\n')


# Определим, какие решения рациональны
def rationals(solutions):
    rationals = []
    for solution in solutions:
        if all(sol.is_rational for sol in solution):
            rationals.append(solution)
    return rationals

print('\nРациональные решения системы:')
print(rationals(solutions))

# И найдём наименьшее поле k
from sympy import QQ, RR

def min_field(solutions):
    extension = []
    field = QQ
    for solution in solutions:
        if field == QQ:
            if not all(sol.is_rational for sol in solution):
                if not all(sol.is_real for sol in solution):
                    field = RR
                    extension = []
                    for sol in solution:
                        if not sol.is_real:
                            extension.append((abs(sol).primitive())[1])
                else:
                    for sol in solution:
                        if not sol.is_rational:
                            extension.append((abs(sol).primitive())[1])
        else:
            if not all(sol.is_real for sol in solution):
                for sol in solution:
                    if not sol.is_real:
                        extension.append((abs(sol).primitive())[1])
    return ('{0}('.format(field) + str(set(extension)) + ')').replace('{', '').replace('}', '')

print('Наименьшее поле k такое, что все решения принадлежат k^2, - это', min_field(solutions))


Все решения системы: 
(-1, -1)
(-sqrt(3), 0)
(sqrt(3), 0)
(1, 1)

Рациональные решения системы:
[(-1, -1), (1, 1)]
Наименьшее поле k такое, что все решения принадлежат k^2, - это QQ(sqrt(3))


### Упражнение 4 ###

Рассмотрим теперь систему из трёх уравнений от трёх переменных: $x, y, z$.
$$x^2 + y^2 + z^2 = 4$$
$$x^2 + 2y^2 = 5$$
$$xz = 1$$
Нужно найти базисы исключающих идеалов $I_1, I_2$ идеала $I = (x^2 + y^2 + z^2 - 4, x^2 + 2y^2 - 5, xz - 1)$ и выяснить, сколько рациональных решений имеет эта система.

**Решение**
По теореме об исключении базис исключающего идеала $I_1$ равен $G_1 = G \cap k[y, z]$, а базис исключающего идеала $I_2$ соответственно равен $G_2 = G \cap k[z]$.
Далее решение будет аналогичным (добавится лишь ещё один шаг исключения и ещё один шаг продолжения).

In [65]:
from sympy.abc import z

F = [x**2 + y**2 + z**2 - 4, x**2 + 2*y**2 - 5, x*z - 1]
G = groebner(F, x, y, z, order='lex')

print('\nБазис Грёбнера относительно x > y > z:')
print_groebner(G)



Базис Грёбнера относительно x > y > z:
g1 = x + 2*z**3 - 3*z
g2 = y**2 - z**2 - 1
g3 = 2*z**4 - 3*z**2 + 1


Теперь найдём $G_1$:

In [66]:
poly_system = G.exprs
G2, I1 = first_exclusion_step([z], poly_system, x, y)

print('\nБазис Грёбнера 2-ого исключающего идеала:')
print_groebner_of_ex(G2)



Базис Грёбнера 2-ого исключающего идеала:
g3 = 2*z**4 - 3*z**2 + 1


Найдём частные решения $z$:

In [67]:
roots_z = []
poly_system = []
for pol in G2:
    f_z = Poly(pol[0], z)
    poly_system.append(f_z)
roots_z = solve_poly_system(poly_system, z)
print('корни Z:', roots_z)

for root_z in roots_z:
    for pol in I1:
        f_xy = Poly(pol[0], x, y)
        if root_z not in solve(LC(f_xy), z):
            break
    else:
        roots_z.remove(root_z)
print('корни Z после проверки:', roots_z)


корни Z: [(-1,), (1,), (-sqrt(2)/2,), (sqrt(2)/2,)]
корни Z после проверки: [(-1,), (1,), (-sqrt(2)/2,), (sqrt(2)/2,)]


Найдём $G_1$

In [68]:
poly_system = []
for pol in I1:
    poly_system.append(pol[0].subs(z, roots_z[0][0]))

def exclusion_step(var, poly_system, I_prev, *args):
    G_ex = []
    I_next = []
    index = 1
    for pol in poly_system:
        f = Poly(pol, *var)
        if f.free_symbols_in_domain.isdisjoint(set(args)):
            G_ex.append((I_prev[index - 1][0].as_expr(), index))
        else:
            I_next.append((I_prev[index - 1][0], index))
        index += 1
    return G_ex, I_next

G1, I0 = exclusion_step([y], poly_system, I1, x)

print('\nБазис Грёбнера 1-ого исключающего идеала:')
print_groebner_of_ex(G1)



Базис Грёбнера 1-ого исключающего идеала:
g2 = y**2 - z**2 - 1


Подставим известные частные решения $z$ и получим уравнение от одной переменной $y$:

In [69]:
roots_y = []
for root_z in roots_z:
    poly_system = []
    for pol in G1:
        f_y = Poly(pol[0].as_expr().subs(z, root_z[0]), y)
        poly_system.append(f_y)
    solution = solve_poly_system(poly_system, y)
    roots_y.append(solution)
print('корни Y:', roots_y)

for i in range(len(roots_z)):
    for root_y in roots_y[i]:
        for pol in I0:
            f_x = Poly(pol[0].as_expr().subs({y:root_y, z:roots_z[i][0]}), x)
            if ((root_y, roots_z[i][0])) not in solve(LC(f_x), y, z):
                break
        else:
            roots_y.remove(root)
print('\nкорни Y после проверки:', roots_y, '\n')


корни Y: [[(-sqrt(2),), (sqrt(2),)], [(-sqrt(2),), (sqrt(2),)], [(-sqrt(6)/2,), (sqrt(6)/2,)], [(-sqrt(6)/2,), (sqrt(6)/2,)]]

корни Y после проверки: [[(-sqrt(2),), (sqrt(2),)], [(-sqrt(2),), (sqrt(2),)], [(-sqrt(6)/2,), (sqrt(6)/2,)], [(-sqrt(6)/2,), (sqrt(6)/2,)]] 



У нас есть 4 частных решения $z$ и для каждого мы получили по два частных решения $y$.
Теперь, зная $z$ и $y$, мы можем найти $x$, а значит, все решения системы.

In [70]:
# Найдём значения X:
roots_x = []
for i in range(len(roots_z)):
    for root_y in roots_y[i]:
        poly_system = []
        for pol in I0:
            poly_system.append(Poly(pol[0].as_expr().subs({y:root_y, z:roots_z[i][0]}), x))
        solution = solve_poly_system(poly_system, x)
        roots_x.append(solution)

# Теперь мы знаем все решения системы, "закинем" их красиво в список решений solutions:
solutions = []
index = 0
for i in range(len(roots_z)):
    for j in range(len(roots_y[i])):
        solutions.append((roots_x[index][0][0], roots_y[i][j][0], roots_z[i][0]))
        index += 1
print('Все решения системы: ')
print(*solutions, sep='\n')

# Определим, какие решения рациональны

print('\nРациональные решения системы:')
print(rationals(solutions))

print('Наименьшее поле k такое, что все решения принадлежат k^2, - это', min_field(solutions))


Все решения системы: 
(-1, -sqrt(2), -1)
(-1, sqrt(2), -1)
(1, -sqrt(2), 1)
(1, sqrt(2), 1)
(-sqrt(2), -sqrt(6)/2, -sqrt(2)/2)
(-sqrt(2), sqrt(6)/2, -sqrt(2)/2)
(sqrt(2), -sqrt(6)/2, sqrt(2)/2)
(sqrt(2), sqrt(6)/2, sqrt(2)/2)

Рациональные решения системы:
[]
Наименьшее поле k такое, что все решения принадлежат k^2, - это QQ(sqrt(6), sqrt(2))


Таким образом, ни одно из решений системы не является рациональным.

### Упражнение 7 ###

Рассмотрим систему
$$t^2 + x^2 + y^2 + z^2 = 0$$
$$t^2 + 2x^2 - xy - z^2 = 0$$
$$t + y^3 - z^3 = 0$$
Мы хотим исключить $t$.

$I = (t^2 + x^2 + y^2 + z^2, t^2 + 2x^2 - xy - z^2, t + y^3 - z^3)$ - соответствующий идеал.

(a) Используя lex-упорядочение с $t > x > y > z$, найти базис Грёбнера $G$ для $I$, а затем базис $G_1$ для $I \cap k[x, y, z]$.

(b) Используя grevlex-упорядочение, найти базис Грёбнера идеала $I \cap k[x, y, z]$.

(c) Объединить полученный в п.(b) базис с полиномом $t + y^3 - z^3$ и доказать, что это объединение является базисом Грёбнера для идеала $I$ по отношению к упорядочению $>_l$.

**Решение**

Найдём базис Грёбнера $G$ и $G_3$ относительно $lex-$упорядочения с $t > x > y > z$:


In [71]:
from sympy.abc import t

F_lex = [t**2 + x**2 + y**2 + z**2, t**2 + 2*x**2 - x*y - z**2, t + y**3 - z**3]
G_lex = groebner(F_lex, t, x, y, z, order='lex')

print('\nБазис Грёбнера относительно lex-упорядочения с t > x > y > z:')
print_groebner(G_lex)


poly_system = G_lex.exprs
G1_lex, I0_lex = first_exclusion_step([x, y, z], poly_system, t)
    
print('\nБазис Грёбнера 1-ого исключающего идеала:')
print_groebner_of_ex(G1_lex)
print('\n')



Базис Грёбнера относительно lex-упорядочения с t > x > y > z:
g1 = t + y**3 - z**3
g2 = x**2 + y**6 - 2*y**3*z**3 + y**2 + z**6 + z**2
g3 = x*y + y**6 - 2*y**3*z**3 + 2*y**2 + z**6 + 3*z**2
g4 = x*z**6 + 3*x*z**2 - y**11 + 4*y**8*z**3 - 5*y**7 - 5*y**5*z**6 - 3*y**5*z**2 + 10*y**4*z**3 - 5*y**3 + 2*y**2*z**9 + 6*y**2*z**5 - 3*y*z**6 - 7*y*z**2
g5 = y**12 - 4*y**9*z**3 + 5*y**8 + 6*y**6*z**6 + 6*y**6*z**2 - 10*y**5*z**3 + 5*y**4 - 4*y**3*z**9 - 12*y**3*z**5 + 5*y**2*z**6 + 13*y**2*z**2 + z**12 + 6*z**8 + 9*z**4

Базис Грёбнера 1-ого исключающего идеала:
g2 = x**2 + y**6 - 2*y**3*z**3 + y**2 + z**6 + z**2
g3 = x*y + y**6 - 2*y**3*z**3 + 2*y**2 + z**6 + 3*z**2
g4 = x*z**6 + 3*x*z**2 - y**11 + 4*y**8*z**3 - 5*y**7 - 5*y**5*z**6 - 3*y**5*z**2 + 10*y**4*z**3 - 5*y**3 + 2*y**2*z**9 + 6*y**2*z**5 - 3*y*z**6 - 7*y*z**2
g5 = y**12 - 4*y**9*z**3 + 5*y**8 + 6*y**6*z**6 + 6*y**6*z**2 - 10*y**5*z**3 + 5*y**4 - 4*y**3*z**9 - 12*y**3*z**5 + 5*y**2*z**6 + 13*y**2*z**2 + z**12 + 6*z**8 + 9*z**4




Мы получили четыре полинома, один из которых имеет степень 12.

Теперь найдём эти же базисы, используя $grevlex-$упорядочение:


In [82]:
F_grevlex = [t**2 + x**2 + y**2 + z**2, t**2 + 2*x**2 - x*y - z**2, t + y**3 - z**3]
G_grevlex = groebner(F, t, x, y, z, order='grevlex')

print('\nБазис Грёбнера относительно grevlex-упорядочения:')
print_groebner(G_grevlex)


poly_system = G_grevlex.exprs
G1_grevlex, I0_grevlex = first_exclusion_step([x, y, z], poly_system, t)

print('\nБазис Грёбнера 1-ого исключащего идеала:')
print_groebner_of_ex(G1_grevlex)



Базис Грёбнера относительно grevlex-упорядочения:
g1 = x + 2*z**3 - 3*z
g2 = x**2 + 2*z**2 - 3
g3 = y**2 - z**2 - 1
g4 = x*z - 1

Базис Грёбнера 1-ого исключащего идеала:
g1 = x + 2*z**3 - 3*z
g2 = x**2 + 2*z**2 - 3
g3 = y**2 - z**2 - 1
g4 = x*z - 1


**Должно было получиться два полинома =(**